In [ ]:
# Import libraries
import warnings
import pandas as pd
import plotly.express as px
from scipy.stats.mstats import trimmed_var
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

warnings.simplefilter(action="ignore", category=FutureWarning)

# Data Preparation

## Import

In [ ]:
# Read data into Dataframe `df`
df = pd.read_csv("CC GENERAL.csv")
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8950 entries, 0 to 8949
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   CUST_ID                           8950 non-null   object 
 1   BALANCE                           8950 non-null   float64
 2   BALANCE_FREQUENCY                 8950 non-null   float64
 3   PURCHASES                         8950 non-null   float64
 4   ONEOFF_PURCHASES                  8950 non-null   float64
 5   INSTALLMENTS_PURCHASES            8950 non-null   float64
 6   CASH_ADVANCE                      8950 non-null   float64
 7   PURCHASES_FREQUENCY               8950 non-null   float64
 8   ONEOFF_PURCHASES_FREQUENCY        8950 non-null   float64
 9   PURCHASES_INSTALLMENTS_FREQUENCY  8950 non-null   float64
 10  CASH_ADVANCE_FREQUENCY            8950 non-null   float64
 11  CASH_ADVANCE_TRX                  8950 non-null   int64  
 12  PURCHA

,CUST_ID,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
0,C10001,40.900749,0.818182,95.40,0.00,95.4,0.000000,0.166667,0.000000,0.083333,0.000000,0,2,1000.0,201.802084,139.509787,0.000000,12
1,C10002,3202.467416,0.909091,0.00,0.00,0.0,6442.945483,0.000000,0.000000,0.000000,0.250000,4,0,7000.0,4103.032597,1072.340217,0.222222,12
2,C10003,2495.148862,1.000000,773.17,773.17,0.0,0.000000,1.000000,1.000000,0.000000,0.000000,0,12,7500.0,622.066742,627.284787,0.000000,12
3,C10004,1666.670542,0.636364,1499.00,1499.00,0.0,205.788017,0.083333,0.083333,0.000000,0.083333,1,1,7500.0,0.000000,NaN,0.000000,12
4,C10005,817.714335,1.000000,16.00,16.00,0.0,0.000000,0.083333,0.083333,0.000000,0.000000,0,1,1200.0,678.334763,244.791237,0.000000,12


## Explore

In [ ]:
df.drop(columns={"CUST_ID"}, inplace=True)

In [ ]:
# Dataframe statistics
df.describe()

,BALANCE,BALANCE_FREQUENCY,PURCHASES,ONEOFF_PURCHASES,INSTALLMENTS_PURCHASES,CASH_ADVANCE,PURCHASES_FREQUENCY,ONEOFF_PURCHASES_FREQUENCY,PURCHASES_INSTALLMENTS_FREQUENCY,CASH_ADVANCE_FREQUENCY,CASH_ADVANCE_TRX,PURCHASES_TRX,CREDIT_LIMIT,PAYMENTS,MINIMUM_PAYMENTS,PRC_FULL_PAYMENT,TENURE
count,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8950.000000,8949.000000,8950.000000,8637.000000,8950.000000,8950.000000
mean,1564.474828,0.877271,1003.204834,592.437371,411.067645,978.871112,0.490351,0.202458,0.364437,0.135144,3.248827,14.709832,4494.449450,1733.143852,864.206542,0.153715,11.517318
std,2081.531879,0.236904,2136.634782,1659.887917,904.338115,2097.163877,0.401371,0.298336,0.397448,0.200121,6.824647,24.857649,3638.815725,2895.063757,2372.446607,0.292499,1.338331
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,50.000000,0.000000,0.019163,0.000000,6.000000
25%,128.281915,0.888889,39.635000,0.000000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.000000,1.000000,1600.000000,383.276166,169.123707,0.000000,12.000000
50%,873.385231,1.000000,361.280000,38.000000,89.000000,0.000000,0.500000,0.083333,0.166667,0.000000,0.000000,7.000000,3000.000000,856.901546,312.343947,0.000000,12.000000
75%,2054.140036,1.000000,1110.130000,577.405000,468.637500,1113.821139,0.916667,0.300000,0.750000,0.222222,4.000000,17.000000,6500.000000,1901.134317,825.485459,0.142857,12.000000
max,19043.138560,1.000000,49039.570000,40761.250000,22500.000000,47137.211760,1.000000,1.000000,1.000000,1.500000,123.000000,358.000000,30000.000000,50721.483360,76406.207520,1.000000,12.000000


In [ ]:
# Checking Missing Values
df.isnull().sum() / len(df)

BALANCE                             0.000000
BALANCE_FREQUENCY                   0.000000
PURCHASES                           0.000000
ONEOFF_PURCHASES                    0.000000
INSTALLMENTS_PURCHASES              0.000000
CASH_ADVANCE                        0.000000
PURCHASES_FREQUENCY                 0.000000
ONEOFF_PURCHASES_FREQUENCY          0.000000
PURCHASES_INSTALLMENTS_FREQUENCY    0.000000
CASH_ADVANCE_FREQUENCY              0.000000
CASH_ADVANCE_TRX                    0.000000
PURCHASES_TRX                       0.000000
CREDIT_LIMIT                        0.000112
PAYMENTS                            0.000000
MINIMUM_PAYMENTS                    0.034972
PRC_FULL_PAYMENT                    0.000000
TENURE                              0.000000
dtype: float64

In [ ]:
df.loc[(df['MINIMUM_PAYMENTS'].isnull()==True),'MINIMUM_PAYMENTS']=df['MINIMUM_PAYMENTS'].mean()
df.loc[(df['CREDIT_LIMIT'].isnull()==True),'CREDIT_LIMIT']=df['CREDIT_LIMIT'].mean()

In [ ]:
# Check for duplicate Values
df.duplicated().sum()

0

In [ ]:
# Calculate variance, get 10 largest features
top_ten_var = df.var().sort_values().tail(10)
top_ten_var

CASH_ADVANCE_TRX          4.657580e+01
PURCHASES_TRX             6.179027e+02
INSTALLMENTS_PURCHASES    8.178274e+05
ONEOFF_PURCHASES          2.755228e+06
BALANCE                   4.332775e+06
CASH_ADVANCE              4.398096e+06
PURCHASES                 4.565208e+06
MINIMUM_PAYMENTS          5.431641e+06
PAYMENTS                  8.381394e+06
CREDIT_LIMIT              1.323950e+07
dtype: float64

In [ ]:
# Create horizontal bar chart of `top_ten_var`
fig = px.bar(
    x = top_ten_var,
    y = top_ten_var.index,
    title = "High Variance Features"
)

fig.update_layout(xaxis_title = "Variance", yaxis_title = "Feature")

fig.show()

In [ ]:
# Create a boxplot of `NHNFIN`
fig = px.box(
    data_frame = df,
    x = "CREDIT_LIMIT",
    title = "Distribution of Credit Cards Limits"
    )
fig.update_layout(xaxis_title = "Value [$]")

fig.show()

In [ ]:
# Calculate trimmed variance
top_ten_trim_var = df.apply(trimmed_var, limits=(0.1,0.1)).sort_values().tail(10)
top_ten_trim_var

CASH_ADVANCE_TRX          6.210728e+00
PURCHASES_TRX             8.462990e+01
INSTALLMENTS_PURCHASES    8.415305e+04
MINIMUM_PAYMENTS          1.541693e+05
ONEOFF_PURCHASES          1.582269e+05
PURCHASES                 3.902004e+05
CASH_ADVANCE              6.109811e+05
PAYMENTS                  7.937212e+05
BALANCE                   1.131014e+06
CREDIT_LIMIT              5.255708e+06
dtype: float64

In [ ]:
# Create horizontal bar chart of `top_ten_trim_var`
fig = px.bar(
    x = top_ten_trim_var,
    y = top_ten_trim_var.index,
    title = "High Variance Features"
    )

fig.update_layout(xaxis_title ="Trimmed Variance", yaxis_title="Feature")

fig.show()

In [ ]:
high_var_cols = top_ten_trim_var.tail(5).index.to_list()
high_var_cols

['PURCHASES', 'CASH_ADVANCE', 'PAYMENTS', 'BALANCE', 'CREDIT_LIMIT']

## Split

In [ ]:
X = df[high_var_cols]
print("X shape:", X.shape)
X.head()

X shape: (8950, 5)


,PURCHASES,CASH_ADVANCE,PAYMENTS,BALANCE,CREDIT_LIMIT
0,95.40,0.000000,201.802084,40.900749,1000.0
1,0.00,6442.945483,4103.032597,3202.467416,7000.0
2,773.17,0.000000,622.066742,2495.148862,7500.0
3,1499.00,205.788017,0.000000,1666.670542,7500.0
4,16.00,0.000000,678.334763,817.714335,1200.0


# Build model

## Iterate

In [ ]:
X_summary = X.aggregate(["mean","std"]).astype(int)
X_summary

,PURCHASES,CASH_ADVANCE,PAYMENTS,BALANCE,CREDIT_LIMIT
mean,1003,978,1733,1564,4494
std,2136,2097,2895,2081,3638


In [ ]:
# Instantiate transformer
ss = StandardScaler()

# Transform `X`
X_scaled_data = ss.fit_transform(X)

# Put `X_scaled_data` into DataFrame
X_scaled = pd.DataFrame(X_scaled_data, columns=X.columns)

print("X_scaled shape:", X_scaled.shape)
X_scaled.head()

X_scaled shape: (8950, 5)


,PURCHASES,CASH_ADVANCE,PAYMENTS,BALANCE,CREDIT_LIMIT
0,-0.424900,-0.466786,-0.528979,-0.731989,-0.960433
1,-0.469552,2.605605,0.818642,0.786961,0.688639
2,-0.107668,-0.466786,-0.383805,0.447135,0.826062
3,0.232058,-0.368653,-0.598688,0.049099,0.826062
4,-0.462063,-0.466786,-0.364368,-0.358775,-0.905464


In [ ]:
X_scaled_summary = X_scaled.aggregate(["mean","std"]).astype(int)
X_scaled_summary

,PURCHASES,CASH_ADVANCE,PAYMENTS,BALANCE,CREDIT_LIMIT
mean,0,0,0,0,0
std,1,1,1,1,1


In [ ]:
n_clusters = range(2,13)
inertia_errors = []
silhouette_scores = []

# Add `for` loop to train model and calculate inertia, silhouette score.
for k in n_clusters:
    #Build model
    model = make_pipeline(
        StandardScaler(),
        KMeans(n_clusters=k, random_state=42)
        )
    #Train model
    model.fit(X)

    #Calculate inertia
    inertia_errors.append(model.named_steps['kmeans'].inertia_)

    #Calculate silhouette score
    silhouette_scores.append(
        silhouette_score(X, model.named_steps['kmeans'].labels_)
        )


print("Inertia:", inertia_errors[:3])
print()
print("Silhouette Scores:", silhouette_scores[:3])

Inertia: [31229.30242026945, 25753.568706692993, 21904.80627378999]

Silhouette Scores: [0.5493067534212036, 0.49545339522172394, 0.43590944455724895]


In [ ]:
# Create line plot of `inertia_errors` vs `n_clusters`
fig = px.line(
    x=n_clusters, y=inertia_errors, title="K-Mean Model: Inertia vs Number of Clusters"
    )

fig.update_layout(xaxis_title="Number of Clusters", yaxis_title="Inertia")

fig.show()

In [ ]:
# Create a line plot of `silhouette_scores` vs `n_clusters`
fig = px.line(
    x=n_clusters, y=silhouette_scores, title="K-Means Model: Silhouette Score vs Number of Clusters"
    )
fig.update_layout(xaxis_title="Number of Clusters", yaxis_title="Silhouette score")
fig.show()

In [ ]:
final_model = make_pipeline(
    StandardScaler(),
    KMeans(n_clusters=3, random_state=42)
)
final_model.fit(X)

Pipeline(steps=[('standardscaler', StandardScaler()),
                ('kmeans', KMeans(n_clusters=3, random_state=42))])

# Results

In [ ]:
labels = final_model.named_steps["kmeans"].labels_
print(labels[:5])

[2 0 2 2 2]


In [ ]:
xgb = X.groupby(labels).mean()
xgb

,PURCHASES,CASH_ADVANCE,PAYMENTS,BALANCE,CREDIT_LIMIT
0,1407.832604,3064.835456,3271.959417,4509.618327,9065.326799
1,11083.715591,5111.941431,18150.407047,4720.766695,12378.740157
2,731.556016,425.493106,1088.194476,830.609660,3302.616848


In [ ]:
# Create side-by-side bar chart of `xgb`
fig = px.bar(
    xgb,
    barmode="group",
    title="Mean Household Finances by Cluster"
)
fig.update_layout(xaxis_title="Cluster", yaxis_title="Value [$]")
fig.show()

In [ ]:
# Instantiate transformer
pca = PCA(n_components=2, random_state=42)

# Transform `X`
X_t = pca.fit_transform(X)

# Put `X_t` into DataFrame
X_pca = pd.DataFrame(X_t, columns= ["PC1","PC2"])

print("X_pca shape:", X_pca.shape)
X_pca.head()

X_pca shape: (8950, 2)


,PC1,PC2
0,-4254.981747,733.218339
1,4533.422706,-149.834778
2,1648.101022,-2768.622856
3,1336.657807,-2741.260915
4,-3660.627187,779.205904


In [ ]:
# Create scatter plot of `PC2` vs `PC1`
fig = px.scatter(
    data_frame=X_pca,
    x= "PC1",
    y="PC2",
    color=labels.astype(str),
    title= "PCA Representation of Clusters"
)
fig.update_layout(xaxis_title="PC1", yaxis_title="PC2")

fig.show()